In [1]:
from genjax.delayed import delay, assume, observe
from genjax import normal
from jax import make_jaxpr
import jax.random as jrand

In [2]:
def fn(obs):
    x = assume(normal, 0.0, 1.0)
    v = observe(obs, normal, 0.0, 1.0)
    return x, v
    
v = delay(fn)(jrand.key(2), 3.0)

KeyError: 298